<div style="direction:rtl; font-family:B Nazanin; font-size:20px"> 
<center>
<h1> بسم الله الرحمن الرحیم </h1>
</center>
<br/>
 این فایل تلاشی است برای استخراج گراف روات صحیح بخاری.
 
 در قسمت زیر نیازهای برنامه import شده‌اند
 به علاوه اینکه مقداری کد تست نیز در آن قرار داد.
 
 قسمت اصلی برنامه از بلاک بعدی آغاز می‌شود.
</div>



In [11]:
%matplotlib inline

import io
import sys
from bs4 import BeautifulSoup
import codecs
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from pymongo import MongoClient

reload(sys)
sys.setdefaultencoding("utf-8")

client = MongoClient()
db = client.rawy_db
collection = db.rawy_collection
out_file = "bokhari_graph.gpickle"


gx = nx.DiGraph()
gx.add_node(u'1')
for i in xrange(2):
    hadithes=[]
    hadithes.append(i)
    gx.add_node(i, hadithes=hadithes)
nx.set_node_attributes(gx, 'hadithes', {u'1': [23432, 234234]})
gx.add_node(u'1', name='ali')
attr = nx.get_node_attributes(gx, 'hadithes')
gx.add_edge(u'1', u'2', count=12)
print "Get node attributes:", attr
print "Get node data:", gx.nodes()
print gx.has_edge(u'2', u'1')
print gx.has_edge(u'1', u'2')
print nx.get_edge_attributes(gx, 'count')
for edge in  gx.edges(data=True):
    print edge
    print "{}-{}".format(edge[0], edge[1])
# nx.draw(gx)

<div style="direction:rtl; font-family:B Nazanin; font-size:20px"> 
<h1>تشکیل گراف با استفاده از لیست احادیث islamwb </h1>
<br/>
در این قسمت مستندات مربوط به هر بخش را بنویسید.

</div>



In [ ]:
g = nx.DiGraph()
debug = False
if debug:
    start = 7020
else:
    start = 1

for hadith in xrange(start, 7032):
    if hadith % 1000 == 0:
        print "Processing hadith: {}".format(hadith)
    html = open('/home/ali/dl/datasets/mine/islamweb/bokhari/{}.html'.format(hadith), 'r').read();
    soup = BeautifulSoup(html, 'html.parser');
    tafseer = soup.find(id='tafseer0')
    tag_names = [child.name for child in tafseer.p.findChildren(recursive=False)]
    font_tag_index = tag_names.index('font')
    last_br_index = len(tag_names) - tag_names[::-1].index('br')
    tags = tafseer.p.findChildren(recursive=False)[last_br_index:font_tag_index]
    
    rawy_ids = [tag.get('href').split('=')[1] for tag in tags]
    links = [(rawy_ids[i], rawy_ids[i+1]) for i in xrange(len(rawy_ids) - 1)]
    g_hadith_attr = nx.get_node_attributes(g, 'hadithes')
    g_edge_count_atter = nx.get_edge_attributes(g, 'count')
    for tag in tags:
        rawy_id = tag.get('href').split('=')[1]
        rawy_name = tag.text
        rawy_hadithes = []
        if rawy_id in g_hadith_attr:
            rawy_hadithes = g_hadith_attr[rawy_id]
        rawy_hadithes.append(hadith)
        g.add_node(rawy_id, hadithes=rawy_hadithes, name=rawy_name)
    
    for link in links:
        if link in g_edge_count_atter:
            count = g_edge_count_atter[link] + 1
        else:
            count = 1
        g.add_edge(link[0], link[1], count=count)
    
# gpickle preservce single element lists while gml and gexf doesn't
nx.write_gpickle(g, out_file)
if debug:
    print nx.get_edge_attributes(g, 'count')

<div style="direction:rtl; font-family:B Nazanin; font-size:20px"> 
<h1> اضافه کردن تعداد احادیث</h1>
<br/>
در این قسمت تعداد احادیثی که هر راوی در سلسله سند آن وجود دارد به ویژگی‌های هر گره اضافه می‌شود.
</div>



In [ ]:
# 0 - Load raw graph from file
g=nx.read_gpickle(out_file)
attr = nx.get_node_attributes(g, 'hadithes')

# 1- Add Hadith Count
hadith_count = {}
for n in attr:
    hadith_count[n] = len(attr[n])
nx.set_node_attributes(g, 'HadithCount', hadith_count)

# 2- Add rawy class to graph
rawy_class = {}
for node in g.nodes():
    rawy = collection.find_one({'id':node})
    if rawy == None:
        print "E: rawy not found", node
    else:
        try:
            rc = int(rawy['class'])
        except ValueError:
            rc = -1
            print 'E: rawy class is incorrent', node
        rawy_class[node] = rc
nx.set_node_attributes(g, 'RawyClass', rawy_class)

# x- Save graph
nx.write_gml(g, 'narrator_count.gml')
nx.write_gpickle(g, out_file)

# Some rawies doesn't find in data base, here is the list of them
# Rawy not found: 5621
# Rawy not found: 17772
# Rawy not found: 22144


<div style="direction:rtl; font-family:B Nazanin; font-size:20px"> 
<h1> اضافه کردن موقعیت هر راوی</h1>
<br/>
 در این قسمت موقعیت مکانی هر راوی در گراف اضافه می‌شود.
</div>



In [14]:
g=nx.read_gpickle(out_file)

rawy_classes = nx.get_node_attributes(g, 'RawyClass')
hadith_count = nx.get_node_attributes(g, 'HadithCount')
edge_count = nx.get_edge_attributes(g, 'count')

edge_ids = {}
for edge in g.edges():
    edge_ids[edge] = "{}-{}".format(edge[0], edge[1])
nx.set_edge_attributes(g, "id", edge_ids)
# print g.edges(data=True)

min_class = min(rawy_classes.values())
max_class = max(rawy_classes.values())
print max_class
print min_class

rawies_in_class = {}
node_x = {}
node_y = {}
vspace = 10
hspace = 50
y_base = 600
for i in xrange(min_class, max_class + 1):
    rawies_in_class[i] = [(k, hadith_count[k]) for k, v in rawy_classes.items() if v == i]
    
    # Bell sort rawies in each class base on hadith count
    rawies_in_class[i].sort(key=lambda x: x[1])
    subarray = [rawies_in_class[i][j::2] for j in xrange(2)]
    subarray[1].reverse()
    rawies_in_class[i] = sum(subarray, [])
    
    # first node position
    first_y = y_base - ((len(rawies_in_class[i]) / 2) * vspace)
    
    # set node position
    for node_count in enumerate(rawies_in_class[i]):
        node = node_count[1][0]
        node_x[node] = i * hspace
        node_y[node] = first_y + node_count[0] * vspace
    
nx.set_node_attributes(g, 'x', node_x)
nx.set_node_attributes(g, 'y', node_y)

# -----------------------------------------------
# Add colors
# https://stackoverflow.com/questions/15140072/how-to-map-number-to-color-using-matplotlibs-colormap
# -----------------------------------------------
node_norm = mpl.colors.Normalize(vmin=min(hadith_count.values()), 
                            vmax=max(hadith_count.values()))
edge_norm = mpl.colors.Normalize(vmin=min(edge_count.values()), 
                            vmax=max(edge_count.values()))
node_map = cm.Greens
edge_map = cm.Purples

node_value2color = cm.ScalarMappable(norm=node_norm, cmap=node_map)
edge_value2color = cm.ScalarMappable(norm=edge_norm, cmap=edge_map)

# https://stackoverflow.com/questions/18337407/saving-utf-8-texts-in-json-dumps-as-utf8-not-as-u-escape-sequence
# data = json_graph.node_link_data(g)  # I couldn't understand file format
with io.open('graph.json', 'w', encoding='utf-8') as json_file:
    header = """
    {
        "nodes" : [
    """
    json_file.write(unicode(header))
    for node in g.nodes(data=True):
        color = node_value2color.to_rgba(node[1]['HadithCount'], bytes=True)
        try: 
            data = """
            {{
                "id": "{}",
                "label" : "{}",
                "size" : {},
                "x": {},
                "y": {},
                "color": "rgba({}, {}, {}, {})"
            }},\n""".format(node[0],
                            node[1]['name'].encode('utf-8'), 
                            node[1]['HadithCount'], 
                            node[1]['x'],
                            node[1]['y'],
                            color[0], color[1], color[2], color[3]
                          )
            json_file.write(unicode(data))
        except:
            print node
            print "E: "
    json_file.write(unicode("""
        ],
        "edges" :[
    """))
    for edge in g.edges(data=True):
        color = edge_value2color.to_rgba(edge[2]['count'], bytes=True)
        data = """
        {{
            "id" : "{}",
            "source": "{}",
            "target": "{}",
            "size": {},
            "color": "rgba({}, {}, {}, {})"
        }},
        """.format("{}-{}".format(edge[0], edge[1]),
                   edge[0],
                   edge[1],
                   edge[2]['count'],
                   color[0], color[1], color[2], color[3])
        json_file.write(unicode(data))
    footer = """
        ]
    }
    """
    json_file.write(unicode(footer))
        

<div style="direction:rtl; font-family:B Nazanin; font-size:20px"> 
<h1>
وارد کردن داده‌های هر راوی در دیتابیس
</h1>
<br/>

</div>



In [ ]:
key2en = {
u'الموالي' : u'Mawali',
u'سنة الميلاد' : u'year_of_birth',
u'النشاط' : u'activity',
u'عمر الراوي' : u'age_of_transmitter',
u'الطبقة' : u'class',
u'الوصف' : u'description',
u'النسب' : u'descent',
u'الرتبة' : u'rank',
u'الأقرباء' : u'relatives',
u'اسم الشهرة' : u'nickname_ism',
u'اسم الراوي' : u'transmitter_name',
u'بلد الوفاة' : u'country_of_death',
u'روى له' : u'narrated_him',
u'الكنية' : u'nickname',
u'اللقب' : u'title',
u'النوع' : u'type',
u'سنة الوفاة' : u'year_of_death',
u'رقم الراوي' : u'id',
u'المذهب' : u'doctrine',
u'الإقامة' : u'residence'
}

def insert_rawy2db(rawy_id):
    html = open('/home/ali/dl/datasets/mine/islamweb/rawy/{}.html'.format(rawy_id), 'r').read();
    soup = BeautifulSoup(html, 'html.parser');
    table = soup.find(id='table1')
    rawy = {}
    for row in table.find_all('tr'):
        for col in enumerate(row.find_all('td')):
            if col[0] % 2 == 0:
                key = col[1].text
            else:
                try:
                    value = col[1].find('input').get('value')
                except:
                    value = col[1].text
                rawy[key2en[key.strip()]] = value.strip()
    collection.insert_one(rawy)

for rawy_id in xrange(1,9281):
    insert_rawy2db(rawy_id)

In [ ]:
# Add exceptional data
for rawy_id in [1131, 5621, 17772, 22144]:
    insert_rawy2db(rawy_id)

In [ ]:
# استخراج نظر رجالی در مورد یک راوی
from bs4 import BeautifulSoup
import codecs
fn_path = "/home/ali/dl/datasets/mine/islamweb/rawy/garh_5815.html"

with codecs.open(fn_path, encoding='windows-1256') as f:
    content = f.read()
    soup = BeautifulSoup(content.encode('utf-8'), "html")
    rows = soup.find_all('tr')
    for row in rows[1:]:
        cols = row.find_all('td')
        row_id = cols[0].find('strong').text
        crit_name = cols[1].find('strong').text
        status = cols[2].find('strong').text
        print crit_name, "\t", status
        
    